# Aula 7 - Exercício - Agregação com Pandas

In [ ]:
# #@title Init Cell - Execute-me!!!
pkg = !pip list | grep 'ipynb-autograde'
if not pkg or pkg[0].replace('  ','').split(' ')[1]!='0.0.35':
  !pip install ipynb_autograde==0.0.35
%env  COURSE python_tribosgd
from ipynb_autograde.main import init_log, validate
init_log()
email_result=!gcloud config get-value account
if not email_result or 'unset' in email_result[0]:
  !gcloud auth login


In [2]:
# Faca o import das bibliotecas
import pandas as pd
import numpy as np

### 7.1 Escreva uma função para ler os dados de PIB e população dos datasets disponibilizados e retornar um dataframe com a agregação por Estados e as métricas soma, média, desvio padrão e mediana do PIB de 2017 e da população de todos os Estados brasileiros. O arredondamento é opcional.

#### As 3 primeiras linhas do resultado esperado:
```
    pop2017                                            2017                
        sum          mean   median            std       sum          mean     median           std  
uf                                                                          
AC   829619  37709.954545  17903.5   79035.975649  14271061  6.486846e+05   229775.0  1.695167e+06  
AL  3375823  33096.303922  17900.5  103131.981409  52843471  5.180732e+05   175900.5  2.184281e+06  
AM  4063614  65542.161290  24438.5  267475.531426  93204173  1.503293e+06   205869.5  9.261592e+06  
```
#### Ou
```
    pop2017                                  2017                          
        sum     mean   median       std       sum       mean    median   std  
uf                                                                       
AC   829619  37710.0  17903.5   79036.0  14271061   648684.6  229775.0   1695166.9
AL  3375823  33096.3  17900.5  103132.0  52843471   518073.2  175900.5   2184280.9
AM  4063614  65542.2  24438.5  267475.5  93204173  1503293.1  205869.5   9261592.2
```

In [3]:
# Faca aqui seus testes do Exercício 7.1
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'



In [4]:
# Faca aqui sua função do Exercício 7.1
def aggregate_pib_pop(path_pop, path_pib):
    """retorne um dataframe com a agregação por Estados e as métricas soma, média, mediana
       e o desvio padrão do PIB de 2017 e da população de todos os Estados brasileiros.
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']
    df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, dtype={'Unnamed: 1': str})
    df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                           'Unnamed: 2': 'municipio'}, inplace=True)
    df_pib_pop = pd.merge(df_pop, df_pib, on='cod_ibge7')
    return df_pib_pop.groupby('uf').agg({'pop2017' : ['sum', 'mean', 'median', 'std'], '2017' : ['sum', 'mean', 'median', 'std']})
    ### END SOLUTION


In [5]:
# Faca aqui a validacao do Exercicio 7.1
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'

test_data = {('pop2017', 'sum'): 829619.0,
 ('pop2017', 'mean'): 37710.0,
 ('pop2017', 'median'): 17903.5,
 ('pop2017', 'std'): 79036.0,
 ('2017', 'sum'): 14271061.0,
 ('2017', 'mean'): 648684.6,
 ('2017', 'median'): 229775.0,
 ('2017', 'std'): 1695166.9}

entradas = [[path_pop, path_pib]]
saidas = [test_data]
validate(aggregate_pib_pop, entradas, lambda x: x.loc['AC'].apply(lambda x: round(x,1)).to_dict(), saidas, "7.1")


(True, 'Parabéns!')

### 7.2 Escreva uma função para agregar o dataset do Titanic por classe (pclass) e sexo (sex) e calcular a soma da coluna survived (sobrevivente) e a média e mediana da coluna fare (preço do ticket). O arredondamento para uma casa decimal é opcional.
#### Dica: Faça a agregação usando um dicionário que mapeia a coluna e a(s) respectiva(s) métrica(s) de agregação solicitada(s) no enunciado.
#### Resultado esperado:
```
              survived        fare          
                   sum        mean    median
pclass sex                                  
1      female      139  109.412385  80.92915
       male         61   69.888385  49.50420
2      female       94   23.234827  23.00000
       male         25   19.904946  13.00000
3      female      106   15.324250  10.48960
       male         75   12.415462   7.89580
```
#### Ou
```
              survived   fare          
                   sum   mean median
pclass sex                                  
1      female      139  109.4   80.9
       male         61   69.9   49.5
2      female       94   23.2   23.0
       male         25   19.9   13.0
3      female      106   15.3   10.5
       male         75   12.4    7.9
```

In [ ]:
# Faca aqui seus testes do Exercício 7.2
path_titanic = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/titanic/titanic.xls'



In [6]:
# Faca aqui sua função do Exercício 7.2
def agg_titanic(df_titanic):
    """retorne aagregação do dataframe df_titanic por pclass e sex e calcular a soma 
       da coluna survived e a média e mediana da coluna fare.
    """
    ### BEGIN SOLUTION
    return df_titanic.groupby(['pclass', 'sex']).agg({'survived': 'sum', 'fare': ['mean', 'median']})
    ### END SOLUTION


In [7]:
# Faca aqui a validacao do Exercicio 7.2
path_titanic = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/titanic/titanic.xls'
df_titanic = pd.read_excel(path_titanic)

test_data = {('survived', 'sum'): {(1, 'female'): 139,  (1, 'male'): 61,
  (2, 'female'): 94,  (2, 'male'): 25,  (3, 'female'): 106,  (3, 'male'): 75},
  ('fare', 'mean'): {(1, 'female'): 109.4,  (1, 'male'): 69.9,  (2, 'female'): 23.2,
  (2, 'male'): 19.9,  (3, 'female'): 15.3,  (3, 'male'): 12.4}, 
  ('fare', 'median'): {(1, 'female'): 80.9,  (1, 'male'): 49.5,  
  (2, 'female'): 23.0,  (2, 'male'): 13.0,  (3, 'female'): 10.5,  (3, 'male'): 7.9}}


entradas = [[df_titanic]]
saidas = [test_data]
validate(agg_titanic, entradas, lambda x: x.apply(lambda x: round(x, 1)).to_dict(), saidas, "7.2")

(True, 'Parabéns!')

### 7.3 Escreva uma função para realizar o agrupamento do dataset de morbidade por UF e por município aplicando a soma sobre as colunas Valor_total, Internações e Dias_permanência. Em seguida calcule, para todos os municípios nesta agregação, o custo médio diário das internações com a fórmula abaixo. O arredondamento é opcional. 
#### Dica: Faça a agregação usando um dicionário que mapeia a coluna e a respectiva métrica de agregação solicitada no enunciado.
#### Use a seguinte fórmula para calculár o custo diário médio de internação:
```python 
custo_medio_diario_intern = Valor_total/Dias_permanência
```

#### As 3 primeiras linhas do resultado esperado:
```
                 Valor_total  Internações  Dias_permanência   custo_medio_diario_intern
uf municipio                                                  
AC Acrelândia      579340.95        643.0            2805.0   206.538663 
   Assis Brasil    418280.89        669.0            2518.0   166.116319 
   Brasiléia      1814919.82       2118.0            8349.0   217.381701
```

#### Ou
```
                 Valor_total  Internações  Dias_permanência   custo_medio_diario_intern  
uf municipio                                                  
AC Acrelândia       579341.0        643.0            2805.0   206.5
   Assis Brasil     418280.9        669.0            2518.0   166.1 
   Brasiléia       1814919.8       2118.0            8349.0   217.4  
```

In [ ]:
# Faca aqui seus testes do Exercício 7.3
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'



In [8]:
# Faça aqui sua função do Exercício 7.3
def custo_medio_internacao_agregado(df_morbidade_pop):
    ### BEGIN SOLUTION
    df_internacoes = df_morbidade_pop.groupby(['uf','municipio']).agg({'Valor_total': 'sum', 'Internações': 'sum', 'Dias_permanência': 'sum'})
    df_internacoes['custo_medio_diario_intern'] = df_internacoes['Valor_total']/df_internacoes['Dias_permanência']
    return df_internacoes
    ### END SOLUTION

In [9]:
# Faca aqui a validacao do Exercicio 7.3
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'

test_data = {'Valor_total': 1534117.3,
 'Internações': 1094.0,
 'Dias_permanência': 5291.0,
 'custo_medio_diario_intern': 289.9}

df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
df_pop['cod_ibge6'] = df_pop['cod_uf'] + df_pop['cod_munic'].apply(lambda x: x[:-1])
df_morbidade = pd.read_csv(path_morbidade, sep=';', decimal=',', dtype={'cod_ibge6': str}, 
                           na_values=['-', '...'])
df_pop_morbidade = pd.merge(df_pop, df_morbidade, on='cod_ibge6') 

entradas = [[df_pop_morbidade]]
saidas = [test_data]
validate(custo_medio_internacao_agregado, entradas, lambda x: x.loc['GO','Abadia de Goiás'].apply(lambda x: round(x, 1)).to_dict(), saidas, "7.3")

(True, 'Parabéns!')

### 7.4 Crie uma função python para retornar a mediana da coluna Valor_total de 5 municípios do dataset de Morbidade do Datasus (ao longo do período representado no dataset disponibilizado). Os municípios de interesse são aqueles que obtiverem o maior custo médio diário de internação em todo o brasil, nesse período. 
#### Esta métrica de custo médio diário de internação foi calculada no exercício 9.3. 
#### Dica 1: Escreva algumas sentenças de código usando operações do pandas para encontrar quais são esses 5 municípios.
#### Dica 2: Depois de encontrar o codigo IBGE desses 5 municípios, faça um filtro no dataframe dos dados de morbidade.
#### Dica 3: Calcule as medianas das séries referentes a cada município, utilizando uma operação de groupby.

#### Resultado esperado:
```
                                 Valor_total
municipio             cod_ibge6             
Campina Grande do Sul 410400      2579493.47
Cardoso Moreira       330115       167867.62
Italva                330205       147509.47
Pedrinhas Paulista    353715        30360.03
Taquaral              355365        15871.31

``` 

#### Ou:
```
               municipio cod_ibge6  Valor_total
0  Campina Grande do Sul    410400   2579493.47
1        Cardoso Moreira    330115    167867.62
2                 Italva    330205    147509.47
3     Pedrinhas Paulista    353715     30360.03
4               Taquaral    355365     15871.31
```

In [11]:
# Faca aqui seus testes do Exercício 7.4
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'



In [12]:
# Faca aqui sua função do Exercício 7.4
def median_top5(path_morbidade, path_pop):
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge6'] = df_pop['cod_uf'] + df_pop['cod_munic'].apply(lambda x: x[:-1])
    df_morbidade = pd.read_csv(path_morbidade, sep=';', decimal=',', dtype={'cod_ibge6': str}, 
                               na_values=['-', '...'])
    df_pop_morb =  pd.merge(df_pop, df_morbidade, on='cod_ibge6')
    grouped=df_pop_morb.groupby(['municipio', 'cod_ibge6'])
    df_final = grouped.agg({'Valor_total':'sum','Internações':'sum','Dias_permanência':'sum'})
    df_final['custo_medio_diario_intern'] = df_final['Valor_total']/df_final['Dias_permanência']
    df_final.sort_values(by=['custo_medio_diario_intern'],ascending=False, inplace=True)
    df_final = df_final.iloc[:5]
    selecao = df_final.reset_index(level='cod_ibge6')['cod_ibge6']
    df_selecao = df_pop_morb[df_pop_morb['cod_ibge6'].isin(selecao)]
    return df_selecao[['Valor_total', 'municipio', 'cod_ibge6']].groupby(['municipio', 'cod_ibge6']).median()

In [13]:
# Faca aqui a validacao do Exercicio 7.4
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'
test_data = {'Valor_total': {0: 2579493.47, 1: 167867.62, 2: 147509.47, 3: 30360.03, 4: 15871.31}, 
              'cod_ibge6': {0: '410400', 1: '330115', 2: '330205', 3: '353715', 4: '355365'}, 
              'municipio': {0: 'Campina Grande do Sul', 1: 'Cardoso Moreira', 2: 'Italva', 3: 'Pedrinhas Paulista', 4: 'Taquaral'}}


entradas = [[path_morbidade, path_pop]]
saidas = [test_data]
val1, m = validate(median_top5, entradas, lambda x: x.reset_index()[['municipio', 'cod_ibge6', 'Valor_total', ]].to_dict(), saidas, "7.4")
print(m)

Parabéns!


### 7.5 Escreva uma função para retornar um dataframe com os nomes das colunas, o total e a porcentagem de valores NA (de cada coluna) num dataframe passado como argumento para esta função.
#### Arredonde (com o método round) o resultado para duas casas decimais (o assert vai testar com 2 casas decimais).
#### Dica 1: Use a função concat para concatenar duas séries. Especifique axis=1 para concatenar por colunas. Se quiser dar nome as colunas depois de concatenar, utilize o argumento keys=['Total','Porcentagem']
#### Dica 2: Uma das séries deve conter o total de valores NA, e a outra série a porcentagem de valores NA. Depois disso concatena-as usando a função concat.

In [ ]:
# Faca aqui seus testes do Exercício 7.5
path_titanic = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/titanic/titanic.xls'



In [14]:
# Faca aqui sua função do Exercício 7.5
def missing_percentage(df):
    """Retorna um dataframe com os nomes das colunas, o total e a porcentagem de valores
       NA em cada coluna do dataframe passado como argumento.
    """
    ### BEGIN SOLUTION
    total = df.isnull().sum()
    percent = round(df.isnull().sum()/len(df)*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Porcentagem'])
    ### END SOLUTION

In [15]:
# Faca aqui a validacao do Exercício 7.5
path_titanic = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/titanic/titanic.xls'
df_titanic = pd.read_excel(path_titanic)

test_data = {'Total': {'body': 1188,  'cabin': 1014,  'boat': 823,  'home.dest': 564,
                       'age': 263,  'embarked': 2,  'fare': 1,  'ticket': 0,  'parch': 0,  
                       'sibsp': 0,  'sex': 0,  'name': 0,  'survived': 0,  'pclass': 0},
             'Porcentagem': {'body': 90.76,  'cabin': 77.46,  'boat': 62.87,
                            'home.dest': 43.09,  'age': 20.09,  'embarked': 0.15, 
                            'fare': 0.08,  'ticket': 0.0,  'parch': 0.0,  'sibsp': 0.0,
                            'sex': 0.0,  'name': 0.0,  'survived': 0.0,  'pclass': 0.0}}



entradas = [[df_titanic]]
saidas = [test_data]
validate(missing_percentage, entradas, lambda x: x.to_dict(), saidas, "7.5")

(True, 'Parabéns!')

### 7.6 Escreva uma função para colocar num mesmo dataframe os dados de PIB percapita e índice de gini dos municípios brasieiros. 
#### Retorne os top 10 registros com maiores PIB percapita com as seguintes colunas 'pib_percapita', 'cod_ibge6', '1991', '2000' e '2010'.

#### Sugestão: Responda, baseado em dados desses dois dadaframes, se um PIB percapita significativamente maior, tem ajudado a melhorar o índice de Gini nesses municípios. Comente com o professor e colegas casos concretos ou estatísticas extraídas dos dois dataframes que demonstram seu argumento.

#### Dica 2: Ao ler o arquivo CSV do índice de gini, se preciso, use encoding='utf8' como parâmetro do método read_csv


### Use as funções a seguir. Elas serão úteis na resolução deste exercício.

In [16]:
def merge_pop_pib(path_pop, path_pib):
    """retorne um dataframe resultante do merge dos dados do pib e da população
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']
    df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, dtype={'Unnamed: 1': str})
    df_pib.rename(columns={'Unnamed: 0': 'nivel', 'Unnamed: 1': 'cod_ibge7', 
                           'Unnamed: 2': 'municipio'}, inplace=True)

    return pd.merge(df_pop, df_pib, on='cod_ibge7')
    ### END SOLUTION
    
def pib_percapita(pib_pop):
    """retorne um dataframe contendo o PIB percapita dos municípios.
    """
    ### BEGIN SOLUTION
    pib_pop['pib_percapita'] = pib_pop['2017']/pib_pop['pop2017']
    pib_pop['pib_percapita'] = pib_pop['pib_percapita'].apply(lambda x: round(x, 1))
    return pib_pop
    ### END SOLUTION


In [17]:
# Faca aqui seus testes do Exercício 7.6
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_gini = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/gini/ginibr.csv'
path_pop= 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'


In [18]:
# Faca aqui sua função do Exercício 7.6
def top10_municipios(path_pib, path_gini, path_pop):
    """retorne um dataframe contendo os 10 municipios com maiores PIB percapita. O dataframe deve conter também 
    o valor do índice de gini (1991, 2000 e 2010) e o código ibge do respectivo município.
    """
    ### BEGIN SOLUTION
    pib_pop = merge_pop_pib(path_pop, path_pib)
    pib_percapita_2017 = pib_percapita(pib_pop)
    pib_percapita_2017.loc[:, 'cod_ibge6'] = pib_percapita_2017.loc[:,'cod_ibge7'].apply(lambda x: x[:-1])

    df_gini = pd.read_csv(path_gini, sep=';', skiprows=2, skipfooter=2, encoding='utf8', engine='python', decimal=',', na_values='...')
    df_gini.loc[:, 'cod_ibge6'] = df_gini.loc[:,'Município'].apply(lambda x: x.split(' ')[0])
    df_gini.loc[:, 'Município'] = df_gini.loc[:,'Município'].apply(lambda x: ' '.join(x.split(' ')[1:]))

    pib_gini = pib_percapita_2017.merge(df_gini, on='cod_ibge6', how='outer')
    df_top10 = pib_gini.sort_values(by='pib_percapita', ascending=False).iloc[:10,:]

    return df_top10
    ### END SOLUTION


In [19]:
# Faca aqui a validacao do Exercício 7.6
path_pib = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/pib/pib_municipios.xlsx'
path_gini = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/gini/ginibr.csv'
path_pop='https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'

test_data = {'pib_percapita': 344.8, '1991': 0.5, '2000': 0.6, '2010': 0.5}

entradas = [[path_pib, path_gini, path_pop]]
saidas = [test_data]
validate(top10_municipios, entradas, lambda x: x.iloc[0][['pib_percapita', '1991', '2000', '2010']].apply(lambda x: round(x,1)).to_dict(), saidas, "8.6")

(True, 'Parabéns!')

### 7.7 Construa uma função para concatenar (verticalmente) o conteúdo de vários arquivos CSV do dataset de morbidade. Esses arquivos contêm dados com colunas compatíveis.
#### Extraia a data do cabeçalho do arquivo. Divida a coluna 'Município' em duas outras contendo o código ibge de 6 caracteres e o nome do município.
#### Dica 1: Construa uma lista de arquivos presentes num diretório usando a função glob da biblioteca glob
#### Dica 2: Num loop, leia os arquivo duas vezes. Na primeira vez, pra carregar somente a parte desejada do cabeçalho, onde se encontra o mês/ano do respectivo arquivo, utilize o parâmetro nrows. Preste atenção ao separador dos arquivos CSV, é um ponto e vírgula.
#### Dica 3: O método split pode dividir uma string na posição de um caracter especificado.
#### Dica 4: Use o mapeamento nomes abreviados de meses em português para inglês disponibilizado na célula abaixo. Converta os nomes dos meses para inglês na hora de inseri-los no dataframe. Certifique-se de que está usando a configuração de locale en_US.
#### Dica 5: O código strptime(nome_mes,'%b').tm_mon converte nome do mês no seu respectivo número de 1 a 12
#### Dica 6: Para criar uma data a partir dos seus atributos use um código semelhante ao exemplo a seguir import datetime datetime.date(year=2020, month=1, day=1)
#### Dica 7: Para construir uma nova coluna aplicando uma regra para cada célula use a função apply e construa uma função lambda.
#### Dica 8: O método join de uma string consegue unir, com um caracter separador, o conteúdo de uma lista de  elementos numa string. Exemplo: ' '.join(lista_exemplo), produz uma string concatenando com um espaçao os elementos da lista_exemplo. Pode ser útil para isolar o nome do município.
#### Dica 9: Adicione todos os dataframes construídos dentro do loop a uma lista e depois use o método concat para concatenar os dataframes verticalmente. Use o parâmetro ignore_index=True para ignorar o índice dos dataframes.



In [20]:
# Faca aqui seus testes do Exercício 7.7
# Se precisar, faça o clone do repositorio
!git clone "https://github.com/alexlopespereira/enapespcd2021.git"
# Verifique se os arquivos estão no caminho esperado
!ls ./enapespcd2021/data/originais/morbidade/desagregado/2020/   
dir_path = "./enapespcd2021/data/originais/morbidade/desagregado/2020/"
mapeamento = {"Jan":"Jan", "Fev": "Feb", "Mar": "Mar", 
              "Abr": "Apr", "Mai":"May", "Jun":"Jun", "Jul":"Jul", "Ago":"Aug", 
              "Set":"Sep", "Out":"Oct", "Nov":"Nov", "Dez":"Dec"}


Cloning into 'enapespcd2021'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 79 (delta 37), reused 74 (delta 36), pack-reused 0
Unpacking objects: 100% (79/79), done.
A002126189_28_143_208.csv  A102850189_28_143_208.csv  A212148189_28_143_208.csv
A102654189_28_143_208.csv  A102927189_28_143_208.csv  A212323189_28_143_208.csv
A102744189_28_143_208.csv  A103139189_28_143_208.csv  A212334189_28_143_208.csv
A102812189_28_143_208.csv  A103238189_28_143_208.csv  A212345189_28_143_208.csv


In [21]:
# Faca aqui sua função do Exercício 7.7
def concat_morbidade(dir_path):
    """Retorne um dataframe com a concatenação vertical dos dados dos arquivos do diretorio dir_path
    """
    ### BEGIN SOLUTION
    import glob
    from time import strptime
    import datetime
    ### Se estiver executando num jupyter notebook localmente (no seu PC), ou seja, 
    ### não está usando o Google Colab, então execute as instrucoes das duas próximas linhas:
    # import locale
    # locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    pieces = []
    result = None
    mapeamento = {"Jan":"Jan", "Fev": "Feb", "Mar": "Mar", 
              "Abr": "Apr", "Mai":"May", "Jun":"Jun", "Jul":"Jul", "Ago":"Aug", 
              "Set":"Sep", "Out":"Oct", "Nov":"Nov", "Dez":"Dec"}
    for file in glob.glob(dir_path+"/*.csv"):
        periodo = pd.read_csv(file, encoding='iso8859-1', skiprows=2, nrows=1, sep=";")
        mes_ano = periodo.columns.values[0].split(':')[1]
        mes, ano = mes_ano.split('/')
        frame = pd.read_csv(file, encoding='iso8859-1', skiprows=3, sep=';', skipfooter=7, engine='python')
        frame['mes'] = mapeamento[mes]
        frame['ano'] = ano
        frame['mes_ano'] = datetime.date(year=int(ano), month=strptime(mapeamento[mes],'%b').tm_mon, day=1)
        frame['cod_ibge6'] = frame['Município'].apply(lambda x: x.split(' ')[0]) 
        frame['Município'] = frame['Município'].apply(lambda x: ' '.join(x.split(' ')[1:])) 
        pieces.append(frame)
    # Concatena tudo num único DataFrame
    result = pd.concat(pieces, ignore_index=True)
    
    return result
    ### END SOLUTION


In [39]:
# Faca aqui a validacao do Exercício 7.7
# Os arquivos devem estar nesse caminho
dir_path = "./enapespcd2021/data/originais/morbidade/desagregado/2020/"
test_data1 = {'Valor_médio_intern': '1040,25', 'ano': '2020'}
test_data2 = {'Valor_médio_intern': 1040.25, 'ano': '2020'}
entradas = [[dir_path]]
saidas = [test_data1]

val1 = validate(concat_morbidade, entradas, lambda x: x.set_index(['cod_ibge6']).loc['110001',['Valor_médio_intern','ano']].iloc[0].to_dict(), saidas, "8.7")

if not val1:
    saidas = [test_data2]
    val2 = validate(concat_morbidade, entradas, lambda x: x.set_index(['cod_ibge6']).loc['110001',['Valor_médio_intern','ano']].iloc[0].to_dict(), saidas, "8.7")
    print(val2)
else:
    print(val1)

(True, 'Parabéns!')
